In [7]:
import requests
import os
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import time
#load_dotenv()
# Configuración API
#TOKEN = "Xegy8dVsa1H8SFfojJcwYtDL"
#print(f"TOKEN: {os.getenv('BUK_AUTH_TOKEN')}")

In [ ]:
URL="https://pokeapi.co/api/v2/pokemon/pikachu"
respuesta = requests.get(URL)
respuesta_api = respuesta.json()

print(json.dumps(respuesta_api, indent=4))

In [ ]:
def buscando_pokemones():
    URL="https://pokeapi.co/api/v2/pokemon/"
    data_pokemones  = []
    pagina_actual = 1

    while URL and pagina_actual <= 3:
        print(f"🔍 Consultando la URL") 
        try:
            respuesta = requests.get(URL)
            respuesta.raise_for_status()
            respuesta_api = respuesta.json()
            
            pokemones_info = respuesta_api["count"]
            pokemones_por_pagina = respuesta_api["results"]

            for pokemon in pokemones_por_pagina:
                
                detalle = requests.get(pokemon.get("url")).json()
                
                habilidades = [h["ability"]["name"] for h in detalle["abilities"]]
                tipos = [t["type"]["name"] for t in detalle["types"]]

                pokemon_obtenido = {
                    "nombre": pokemon["name"],
                    "habilidades": habilidades,
                    "tipos": tipos
                } 
                data_pokemones.append(pokemon_obtenido)

            print(f"✅ Se han obtenido {len(data_pokemones)} pokemones de un total de {pokemones_info} pokemones.")
            
            URL = respuesta_api.get("next")
            pagina_actual += 1
            
            time.sleep(0.5) 

        except requests.exceptions.RequestException as e:
            print(f"❌ Error en la petición: {e}")
            break

    print("total pokemones:", len(data_pokemones))  
    return data_pokemones

pokemones = buscando_pokemones()
print(json.dumps(pokemones[:5], indent=4)) 


In [17]:
def primeros_pokemones_con_evolucion():
    URL = "https://pokeapi.co/api/v2/pokemon/"
    data_pokemones = []

    try:
        # 1️⃣ Llamamos a la primera página
        respuesta = requests.get(URL)
        respuesta.raise_for_status()
        respuesta_api = respuesta.json()

        # 2️⃣ Tomamos solo los primeros 5 resultados
        pokemones_por_pagina = respuesta_api["results"][:5]

        for pokemon in pokemones_por_pagina:
            # 👉 Detalle del Pokémon
            detalle = requests.get(pokemon["url"]).json()

            # === HABILIDADES ===
            habilidades = []
            for h in detalle["abilities"]:
                nombre_habilidad = h["ability"]["name"]
                habilidades.append(nombre_habilidad)

            # === TIPOS ===
            tipos = []
            for t in detalle["types"]:
                nombre_tipo = t["type"]["name"]
                tipos.append(nombre_tipo)

            # === EVOLUCIONES ===
            species_url = detalle["species"]["url"]
            species_data = requests.get(species_url).json()

            evolution_url = species_data["evolution_chain"]["url"]
            evolution_data = requests.get(evolution_url).json()

            # Recorremos la cadena evolutiva
            evoluciones = []
            evo_chain = evolution_data["chain"]

            while evo_chain:
                evo_nombre = evo_chain["species"]["name"]
                evoluciones.append(evo_nombre)
                if evo_chain["evolves_to"]:
                    evo_chain = evo_chain["evolves_to"][0]  # tomamos la primera rama
                else:
                    evo_chain = None

            # Guardamos la info
            pokemon_obtenido = {
                "nombre": pokemon["name"],
                "habilidades": habilidades,
                "tipos": tipos,
                "cadena_evolutiva": evoluciones
            }

            data_pokemones.append(pokemon_obtenido)
            time.sleep(0.5)  # descansito para no saturar el servidor

    except requests.exceptions.RequestException as e:
        print(f"❌ Error en la petición: {e}")

    return data_pokemones


# 🚀 Ejemplo de uso
pokemones = primeros_pokemones_con_evolucion()
print(json.dumps(pokemones, indent=4))


[
    {
        "nombre": "bulbasaur",
        "habilidades": [
            "overgrow",
            "chlorophyll"
        ],
        "tipos": [
            "grass",
            "poison"
        ],
        "cadena_evolutiva": [
            "bulbasaur",
            "ivysaur",
            "venusaur"
        ]
    },
    {
        "nombre": "ivysaur",
        "habilidades": [
            "overgrow",
            "chlorophyll"
        ],
        "tipos": [
            "grass",
            "poison"
        ],
        "cadena_evolutiva": [
            "bulbasaur",
            "ivysaur",
            "venusaur"
        ]
    },
    {
        "nombre": "venusaur",
        "habilidades": [
            "overgrow",
            "chlorophyll"
        ],
        "tipos": [
            "grass",
            "poison"
        ],
        "cadena_evolutiva": [
            "bulbasaur",
            "ivysaur",
            "venusaur"
        ]
    },
    {
        "nombre": "charmander",
        "habilidades": [